In [1]:
import boto3, json
from pprint import pprint

In [2]:
def getIncludedRegions(allRegions, excludedRegions):
    return([region for region in allRegions if region not in excludedRegions])

def getIncludedServices(allServices, excludedServices):
    return([service for service in allServices if service not in excludedServices])

In [3]:
def getInstanceIds(reservations):
    instanceIds = []
    for reservation in reservations['Instances']:
        for instance in reservation["Instances"]:
            instanceIds.append(instance['InstanceId'])
    return(instanceIds)

In [4]:
def deleteEC2Instances(includedRegions, ec2Client):
    reservations = ec2client.describe_instances(Filters=[{'Name' : 'instance-state-name', 'Values' : ['running']}])
    instanceIds = getInstanceIds(reservations)
    response = ec2client.terminate_instances(InstanceIds=[instanceIds],DryRun=False)
    return(response)

In [5]:
def getVolumeIds(response):
    volumeIds = []
    for volume in response['Volumes']:
        volumeIds.append(volume["VolumeId"])
    return(volumeIds)

In [6]:
def deleteVolumes(includedRegions, ec2Client):
    response = ec2Client.describe_volumes(Filters=[{'Name' : 'attachment.status', 'Values' : ['detached']}])
    volumeIds = getVolumeIds(response)
    for volumeId in volumeIds:
        ec2Client.delete_volume(VolumeId=volumeId,DryRun=False)

In [7]:
def getAllServices(session):
    return(session.get_available_services())

In [8]:
def getAllRegions(allServices, session):
    serviceDict = {}
    for service in allServices:
        availableRegions = session.get_available_regions(service, partition_name='aws')
        serviceDict[service] = availableRegions
    allRegions = serviceDict['ec2']
    return(allRegions)

In [9]:
def getAllServicesAndRegions(allServices, session):
    serviceDict = {}
    for service in allServices:
        availableRegions = session.get_available_regions(service, partition_name='aws')
        serviceDict[service] = availableRegions
    return(serviceDict)

In [10]:
def getAWSCreds():
    credentials = json.load(open('./credentials.json'))
    awsCreds = credentials['aws']
    return(awsCreds)

In [11]:
accessKey = getAWSCreds()['accessKey']
secretKey = getAWSCreds()['secretKey']

In [12]:
session = boto3.session.Session(aws_access_key_id=accessKey, aws_secret_access_key=secretKey)
allServices = getAllServices(session)
allRegions = getAllRegions(allServices, session)

In [13]:
test = getAllServicesAndRegions(allServices, session)

In [14]:
excludedRegions = ['us-west-1']
excludedServices = ['cloudwatch', 'budgets']

In [15]:
includedRegions = getIncludedRegions(allRegions, excludedRegions)
includedServices = getIncludedServices(allServices, excludedServices)
ec2client = boto3.client('ec2', aws_access_key_id=accessKey, aws_secret_access_key=secretKey)

In [17]:
# deleteEC2Instances(includedRegions=includedRegions, ec2Client=ec2client)
# reservations = ec2client.describe_instances(Filters=[{'Name' : 'instance-state-name', 'Values' : ['running']}], DryRun=False)

In [18]:
reservations = ec2client.describe_instances(Filters=[{'Name' : 'instance-state-name', 'Values' : ['running']}], DryRun=False)

In [19]:
reservations

{'Reservations': [],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 06 Dec 2017 10:02:42 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '0d3f3500-fc87-4d5f-bf8e-719145b086be',
  'RetryAttempts': 0}}